In [1]:
from dash import Dash, html, dcc, Input, Output, callback, dash_table
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
import seaborn as sns
from scipy import stats
plt.rc('font', family='Malgun Gothic')

In [ ]:
#이전에 했던거 복기

In [3]:
df = pd.read_excel('data/교육통계데이터_2015_2022_최종.xlsx') 

df2 = round(df.groupby(['시도', '연도'])['학업중단자_비율'].mean(), 2)

df3 = round(df.groupby(['시도', '연도'])['교사수_비율'].mean(), 2)

df4 = pd.concat([df2, df3], axis=1)

df4 = df4.reset_index() 

df5 = df4.copy()

for i in range(len(df4)):
    df4.loc[i,'비율값'] = df4.loc[i, '학업중단자_비율']
    df4.loc[i,'기준'] = "학업중단자"

for i in range(len(df5)):
    df5.loc[i,'비율값'] = df5.loc[i, '교사수_비율']
    df5.loc[i,'기준'] = "교사수"

df4 = df4.drop(['학업중단자_비율', '교사수_비율'], axis=1)

df5 = df5.drop(['학업중단자_비율', '교사수_비율'], axis=1)

df_graph = pd.concat([df4, df5])

df_table = df[['연도', '시도', '행정구역', '학업중단자_비율', '교사수_비율']]

app = Dash(__name__)

app.layout = html.Div([
    # 제목 
    html.H2(children='전문상담 교사 유무와 학업 중단자의 연관성 분석', style={'textAlign': 'center'}),
    
    # 그래프는 시도를 기준으로 Dropdown하여 보여줌
    # 서울을 먼저 보여줌
    dcc.Dropdown(df_graph['시도'].unique(), '서울', id='dropdown-selection'),
    
    # 전문상담 교사 비율과 학업 중단자의 비율을 paired plot으로 표현 
    html.H3(children='<연도별 전문상담교사와 학업중단자 비율 비교>', style={'textAlign': 'center'}),
    dcc.Graph(figure={}, id='graph-content1'),
    
    # 전문상담 교사 비율과 학업 중단자의 연도별 추세 표현 
    html.H3(children='<연도별 전문상담교사와 학업중단자 추세 비교>', style={'textAlign': 'center'}),
    dcc.Graph(figure={}, id='graph-content2'),
    
    # 테이블은 시도별 업데이트를 하지않고 한번에 전체 시도의 테이블을 출력
    dash_table.DataTable(data=df_table.to_dict('records'), page_size=10) 
])


@app.callback(
    Output("graph-content1", "figure"), 
    Input("dropdown-selection", "value"))

def update_bar_chart(value):
    dff = df_graph[df_graph['시도']==value]
    return px.bar(dff, x="연도", y="비율값", color="기준", barmode="group",
                 labels=dict(비율값="연도별 전문상담교사와 학업중단자 비율"))

@app.callback(
    Output("graph-content2", "figure"), 
    Input("dropdown-selection", "value"))

def update_graph(value):
    dff = df_graph[df_graph['시도']==value]
    return px.line(dff, x="연도", y="비율값",color='기준', symbol='기준',
                  labels=dict(비율값="연도별 전문상담교사와 학업중단자 비율")) 

if __name__ == "__main__":
    app.run_server(debug=True)
